In [1]:
from pymongo import MongoClient

In [2]:
db = MongoClient()['misuse_check_gse']

In [5]:
misues = list(db.noinfo.find({
        '$text': {'$search': 'misuse'}
    }))

In [6]:
misues[:2]

[{'_id': ObjectId('5633cb8fa138570d1787d9cf'),
  'data': {},
  'data_source': 'geo',
  'message': 'Your access to the NCBI website at www.ncbi.nlm.nih.gov has been temporarily blocked due to a possible misuse/abuse situation involving your site. To restore access, please have your\nsystem administrator contact\n\n',
  'url': 'http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE57672'},
 {'_id': ObjectId('5633cb8fa138570d1787d9d0'),
  'data': {},
  'data_source': 'geo',
  'message': 'Your access to the NCBI website at www.ncbi.nlm.nih.gov has been temporarily blocked due to a possible misuse/abuse situation involving your site. To restore access, please have your\nsystem administrator contact\n\n',
  'url': 'http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE57673'}]

In [15]:
bad_ids = [
    m['url']
    for m in misues
]

In [9]:
bad_ids[:10]

['GSE57672',
 'GSE57673',
 'GSE57674',
 'GSE57675',
 'GSE57676',
 'GSE57677',
 'GSE57678',
 'GSE57679',
 'GSE57680',
 'GSE57681']

In [10]:
import sys

In [11]:
sys.path.append('/home/ubuntu/scraper-ng/')

In [12]:
from scraper import cache

In [13]:
import config

In [14]:
c = cache.Cache(config.cache_path, config.cache_path_temporary)

In [17]:
import os

In [24]:
bad_ids[0]

'http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE57672'

In [25]:
c.url_path('http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE22149')

('/data/rawdata/cache/6128/61288c56/61288c56c8abbd87e58f7fc1169c81ef16cbdd7c',
 '/data/rawdata/cache/6128/61288c56')

In [18]:
for i in bad_ids:
    path, _ = c.url_path(i)
    os.remove(path)

In [19]:
bad_accs = [
    i.split('acc=')[1]
    for i in bad_ids
]

In [22]:
len(bad_accs)

1125

In [47]:
'GSE57615' in bad_accs

False

In [23]:
#' --task '.join(bad_accs)

## Importing all new series

In [28]:
import pandas as pd
import numpy as np

In [30]:
new_series_accs = pd.DataFrame([s for s in db.series.find({}, {'_id':0, 'accession': 1})])

In [31]:
main_db = MongoClient()['scraper_test_dev']

In [34]:
main_series_accs = pd.DataFrame([s for s in main_db.series.find({'data_source': 'geo'}, {'_id':0, 'accession': 1})])

In [35]:
main_series_accs.head()

,accession
0,GSE66696
1,GSE66668
2,GSE66677
3,GSE66686
4,GSE66695


In [40]:
_t = (
    main_series_accs
    .rename(columns={'accession': 'main_accession'})
    .merge(new_series_accs, left_on='main_accession', right_on='accession', how='outer')
    
)
_t[_t.accession.isnull()]

,main_accession,accession
57459,GSE74539,NaN


In [46]:
new_series_accs.query("accession == 'GSE61728'")

,accession


In [45]:
main_series_accs.query("accession == 'GSE61728'")

,accession


In [44]:
main_db.series.count({
        'accession': "GSE22149"
    })

1